In [1]:
import lime

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(x, feature_names=list(x), class_names=[0, 1], mode='classification')
exp = explainer.explain_instance(x_test[0], model.predict, num_features=8)
exp.show_in_notebook(show_table=True, show_all=False)


# changed x to x_train
explainer = lime.lime_tabular.LimeTabularExplainer(x_train, feature_names=list(x), class_names=[0, 1], mode='classification')
# added top_labels=1
exp = explainer.explain_instance(x_test[2], model.predict, num_features=8, top_labels=1)